In [17]:
import sys, os
from datetime import datetime, timedelta
import pandas as pd

project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

data_dir = os.path.join(project_root, "data")
os.makedirs(data_dir, exist_ok=True)



In [18]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=800)).date()

spx_df = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir,
    filter_by_date=False

)

print("✓ spx_df loaded:", len(spx_df))
spx_df.head()


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded all 16401 options from CSV (no date filtering)
✓ spx_df loaded: 16401


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker
0,2025-12-16,2025-12-16,2800.0,P,3.234377,0.05,6816.509766,102.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-2800.0-P,SPXW251216P02800000,^SPX
1,2025-12-16,2025-12-16,3000.0,P,2.984378,0.05,6816.509766,10.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-3000.0-P,SPXW251216P03000000,^SPX
2,2025-12-16,2025-12-16,3200.0,P,2.765628,0.25,6816.509766,2.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-3200.0-P,SPXW251216P03200000,^SPX
3,2025-12-16,2025-12-16,3400.0,P,2.546879,0.05,6816.509766,201.0,0.0,0.05,0.0,0.05,50.0,^SPX-2025-12-16-3400.0-P,SPXW251216P03400000,^SPX
4,2025-12-16,2025-12-16,3600.0,P,2.343754,0.05,6816.509766,17.0,0.0,0.05,0.0,0.05,10.0,^SPX-2025-12-16-3600.0-P,SPXW251216P03600000,^SPX


In [19]:
# Cell 2 - Utilisez la date du CSV au lieu de datetime.now()
from datetime import datetime
as_of = datetime(2025, 12, 12)  # Date de création du CSV (12 décembre 2025)
spx_df["expiry_date"] = pd.to_datetime(spx_df["expiry_date"])
spx_df["T"] = (spx_df["expiry_date"] - pd.Timestamp(as_of)).dt.total_seconds() / (365.25 * 24 * 3600)

print(f"Options avec T > 1 an dans les données brutes: {len(spx_df[spx_df['T'] > 1.0])}")
print(f"T max dans les données: {spx_df['T'].max():.4f} ans")

Options avec T > 1 an dans les données brutes: 1001
T max dans les données: 1.5113 ans


In [20]:
from iv_surface_spx import SPXIVSurface, SurfaceConfig

as_of_date = datetime.now()

cfg = SurfaceConfig(
    r=0.05,
    min_bid=0.01,
    max_rel_spread=0.25,
    min_oi=10,
    min_volume=1,
    grid_n=60,
    rbf_smoothing=0.5,
    min_T=1/365,  # Commencer à ~1 jour
    max_T=2.0,     # Aller jusqu'à 2 ans
    as_of=as_of_date
)

spx_surface = SPXIVSurface(spx_df, cfg)
print("Rows after cleaning:", len(spx_surface.df))
spx_surface.df.head()


Rows after cleaning: 7873


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,...,underlying_ticker,T,S,bid,ask,mid,rel_spread,iv_pct,F,x
0,2025-12-18,2025-12-18,6400.0,P,0.252693,0.42,6816.509766,272,0.40,0.50,...,^SPX,0.00539,6816.509766,0.40,0.50,0.450,0.222222,25.269302,6821.903881,-0.063841
1,2025-12-18,2025-12-18,6430.0,P,0.239021,0.60,6816.509766,398,0.45,0.55,...,^SPX,0.00539,6816.509766,0.45,0.55,0.500,0.200000,23.902128,6821.903881,-0.059164
2,2025-12-18,2025-12-18,6475.0,P,0.220711,0.55,6816.509766,130,0.55,0.70,...,^SPX,0.00539,6816.509766,0.55,0.70,0.625,0.240000,22.071092,6821.903881,-0.052190
3,2025-12-18,2025-12-18,6480.0,P,0.217903,0.75,6816.509766,322,0.55,0.70,...,^SPX,0.00539,6816.509766,0.55,0.70,0.625,0.240000,21.790333,6821.903881,-0.051418
4,2025-12-18,2025-12-18,6485.0,P,0.214974,0.70,6816.509766,37,0.55,0.70,...,^SPX,0.00539,6816.509766,0.55,0.70,0.625,0.240000,21.497367,6821.903881,-0.050647


In [21]:
fig = spx_surface.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


### Summary (SPX IV Surface — OTM, forward-moneyness)

- The surface exhibits a **strong downside skew**: implied volatility increases sharply as  
  **log-moneyness** $\ln(K/F)$ becomes more negative (deep OTM puts).  
  This is typical for SPX and reflects **crash-risk insurance demand**.

- Volatility varies with **time to expiry**, and the **term structure depends on moneyness**  
  (the maturity effect is not uniform across strikes).

- The “striped” pattern in the white points is expected because option quotes exist on  
  **discrete expiries and strikes**; the smooth surface is an interpolation across that grid.

- Extremely high IV levels (e.g., **60–70%**) in the far left tail are likely driven by  
  **illiquid/unstable quotes and interpolation extrapolation**, so the outer tail should be  
  treated cautiously (e.g., **outlier filtering** or **restricting the moneyness range** before fitting).


In [22]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=360)).date()

spx_df1 = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)

print("✓ spx_df loaded:", len(spx_df1))
spx_df


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded 15400 options after filtering by date range (2025-12-16 to 2026-12-11)
   (1001 options filtered out)
✓ spx_df loaded: 15400


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker,T
0,2025-12-16,2025-12-16,2800.0,P,3.234377,0.05,6816.509766,102.0,0.00,0.05,0.00,0.05,1.0,^SPX-2025-12-16-2800.0-P,SPXW251216P02800000,^SPX,0.010951
1,2025-12-16,2025-12-16,3000.0,P,2.984378,0.05,6816.509766,10.0,0.00,0.05,0.00,0.05,1.0,^SPX-2025-12-16-3000.0-P,SPXW251216P03000000,^SPX,0.010951
2,2025-12-16,2025-12-16,3200.0,P,2.765628,0.25,6816.509766,2.0,0.00,0.05,0.00,0.05,1.0,^SPX-2025-12-16-3200.0-P,SPXW251216P03200000,^SPX,0.010951
3,2025-12-16,2025-12-16,3400.0,P,2.546879,0.05,6816.509766,201.0,0.00,0.05,0.00,0.05,50.0,^SPX-2025-12-16-3400.0-P,SPXW251216P03400000,^SPX,0.010951
4,2025-12-16,2025-12-16,3600.0,P,2.343754,0.05,6816.509766,17.0,0.00,0.05,0.00,0.05,10.0,^SPX-2025-12-16-3600.0-P,SPXW251216P03600000,^SPX,0.010951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16396,2027-06-17,2027-06-17,10400.0,C,0.155351,8.60,6816.509766,186.0,2.65,7.50,2.65,7.50,1.0,^SPX-2027-06-17-10400.0-C,SPX270617C10400000,^SPX,1.511294
16397,2027-06-17,2027-06-17,10800.0,C,0.160279,5.20,6816.509766,21.0,0.65,5.50,0.65,5.50,10.0,^SPX-2027-06-17-10800.0-C,SPX270617C10800000,^SPX,1.511294
16398,2027-06-17,2027-06-17,11000.0,C,0.162904,2.90,6816.509766,20.0,0.05,4.80,0.05,4.80,20.0,^SPX-2027-06-17-11000.0-C,SPX270617C11000000,^SPX,1.511294
16399,2027-06-17,2027-06-17,11200.0,C,0.165902,2.20,6816.509766,11.0,0.05,4.30,0.05,4.30,1.0,^SPX-2027-06-17-11200.0-C,SPX270617C11200000,^SPX,1.511294


In [23]:
cfg = SurfaceConfig(
    min_T=1/365,  # Min 3-4 days Base 1 year =1, 1 month =1/12 etc.
    max_T=365   #
)
spx_surface_test = SPXIVSurface(spx_df1, cfg)
surface = SPXIVSurface(spx_df, cfg=cfg)

fig = spx_surface_test.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


In [24]:
print(f"Options bf filtering : {len(spx_df1)}")
surface = SPXIVSurface(spx_df1, cfg=cfg)
print(f"Options after filtering: {len(surface.df)}")

Options bf filtering : 15400
Options after filtering: 7349


In [25]:
from datetime import datetime
import pandas as pd

# Convert expiry dates to datetime
spx_df['expiry_date'] = pd.to_datetime(spx_df['expiry_date'])

# Set reference date (as of today)
as_of = datetime.now()

# Calculate time to expiration in years
spx_df['T'] = (spx_df['expiry_date'] - pd.Timestamp(as_of)).dt.total_seconds() / (365.25 * 24 * 3600)

# Display the time range in your data
print(f"T min in your data: {spx_df['T'].min():.3f}")
print(f"T max in your data: {spx_df['T'].max():.3f}")

T min in your data: -0.000
T max in your data: 1.500


In [26]:
# Test avec un intervalle plus court
start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=1000)).date() 

spx_df2 = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)
print(f"Options avec intervalle de 30 jours: {len(spx_df2)}")
# Devrait être moins que 14694

⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded 16401 options after filtering by date range (2025-12-16 to 2028-09-11)
Options avec intervalle de 30 jours: 16401


In [27]:
spx_df2 = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir,
    filter_by_date=False  # ⬅️ Désactive le filtrage par date
)

⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded all 16401 options from CSV (no date filtering)


In [28]:

df_test = spx_df2.copy()
df_test["expiry_date"] = pd.to_datetime(df_test["expiry_date"])
as_of = datetime.now()
df_test["T"] = (df_test["expiry_date"] - pd.Timestamp(as_of)).dt.total_seconds() / (365.25 * 24 * 3600)

# Filtrer T > 1
df_long = df_test[df_test["T"] > 1.0].copy()
print(f"Options avec T > 1 an: {len(df_long)}")

# Vérifier chaque filtre
print(f"\nAprès filtres de qualité:")
print(f"  - Avec bid/ask valides: {len(df_long[(df_long['bid_price'] > 0) & (df_long['ask_price'] > 0)])}")
print(f"  - Avec bid >= 0.01: {len(df_long[df_long['bid_price'] >= 0.01])}")
print(f"  - Avec liquidité (OI>=10 ou Vol>=1): {len(df_long[(df_long['open_interest'] >= 10) | (df_long['volume'] >= 1)])}")
print(f"  - Avec IV valide: {len(df_long[df_long['mark_iv'].notna()])}")

Options avec T > 1 an: 1001

Après filtres de qualité:
  - Avec bid/ask valides: 911
  - Avec bid >= 0.01: 912
  - Avec liquidité (OI>=10 ou Vol>=1): 945
  - Avec IV valide: 1001


In [29]:
cfg = SurfaceConfig(
    min_T=1/365,  # Min 3-4 days Base 1 year =1, 1 month =1/12 etc.
    max_T=3   #
)
spx_surface_test = SPXIVSurface(spx_df2, cfg)
surface = SPXIVSurface(spx_df2, cfg=cfg)

fig = spx_surface_test.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


In [30]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=500)).date()

spx_df_test = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)

print("✓ spx_df loaded:", len(spx_df_test))
spx_df_test.head()


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded 16083 options after filtering by date range (2025-12-16 to 2027-04-30)
   (318 options filtered out)
✓ spx_df loaded: 16083


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker
0,2025-12-16,2025-12-16,2800.0,P,3.234377,0.05,6816.509766,102.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-2800.0-P,SPXW251216P02800000,^SPX
1,2025-12-16,2025-12-16,3000.0,P,2.984378,0.05,6816.509766,10.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-3000.0-P,SPXW251216P03000000,^SPX
2,2025-12-16,2025-12-16,3200.0,P,2.765628,0.25,6816.509766,2.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-16-3200.0-P,SPXW251216P03200000,^SPX
3,2025-12-16,2025-12-16,3400.0,P,2.546879,0.05,6816.509766,201.0,0.0,0.05,0.0,0.05,50.0,^SPX-2025-12-16-3400.0-P,SPXW251216P03400000,^SPX
4,2025-12-16,2025-12-16,3600.0,P,2.343754,0.05,6816.509766,17.0,0.0,0.05,0.0,0.05,10.0,^SPX-2025-12-16-3600.0-P,SPXW251216P03600000,^SPX


In [31]:
cfg = SurfaceConfig(
    min_T=0.5,  # Min 3-4 days Base 1 year =1, 1 month =1/12 etc.
    max_T=2.0   #
)
spx_surface_test = SPXIVSurface(spx_df_test, cfg)
surface = SPXIVSurface(spx_df_test, cfg=cfg)

fig = spx_surface_test.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=False
)
fig.show()

In [32]:
# Vérifier quel CSV est chargé
import glob
csv_files = glob.glob(os.path.join(data_dir, "sp500_options_SPX_*.csv"))
print("CSV trouvés:")
for f in sorted(csv_files):
    print(f"  {f}")
    # Vérifier les dates dans chaque CSV
    df_check = pd.read_csv(f)
    df_check["expiry_date"] = pd.to_datetime(df_check["expiry_date"])
    print(f"    Expiry max: {df_check['expiry_date'].max()}")

# Supprimer les anciens CSV pour forcer le re-téléchargement
for f in csv_files:
    if "20251212" in f:  # Supprimer l'ancien CSV du 12 décembre
        os.remove(f)
        print(f"Supprimé: {f}")

# Puis re-importez
spx_df = import_sp500_options_data(
    start_date=datetime.now().date(),
    end_date=datetime(2027, 6, 17).date(),  # Date max que vous voulez
    ticker="^SPX",
    output_dir=data_dir,
    filter_by_date=False
)

CSV trouvés:
  /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
    Expiry max: 2027-06-17 00:00:00
⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251216_004012.csv
📊 Total rows in CSV: 16401
✅ Loaded all 16401 options from CSV (no date filtering)
